In [42]:
import pandas as pd
import numpy as np
# Set default display precision to 4 decimal places
# pd.set_option('precision', 4)
pd.set_option('display.float_format', lambda x: '%.4f' % x)


# Here, I assume that the data file is in the same directory as the the running
# Python interpeter. This often means the same directory as this code file.
# Change the path variable below to suit your particular setup.
path_to_data_file = 'assetclass_data_monthly_2.xlsx'
df = pd.read_excel(path_to_data_file)
df = df.set_index('Dates')



# Assume that the risk-free rate is 0.01/12 in months without data on Cash's
# returns, and that otherwise it equals the Cash return


In [ ]:
df['rf'] = df['Cash'] #.fillna(0.01/12)
rf_mean_annualized = df['rf'].mean() * 12

risky_assets = df.columns[0:5]
risky_assets = risky_assets#.sort_values()
df_tilde = df.loc[:, risky_assets]
df_tilde = df_tilde.subtract(df['rf'], axis=0)
# Alternative:
# df_tilde = df_tilde.apply(lambda x: x - df['rf'])

## Problem 1 ##

mu_tilde = 12 * df_tilde.mean()
sigma = np.sqrt(12) * df_tilde.std()
table1 = pd.DataFrame({'Mean':mu_tilde, 'Vol':sigma, 'Sharpe':mu_tilde/sigma})


## Problem 1 (a) and (b)

In [43]:
table1

,Mean,Vol,Sharpe
Fund A,0.0987,0.1497,0.6593
Fund B,0.1120,0.1540,0.7270
Fund C,0.0943,0.1666,0.5659
Fund D,0.1473,0.1192,1.2354
Fund E,0.0845,0.0963,0.8781


In [44]:
def compute_tangency(df_tilde, diagonalize_Sigma=False):
    """Compute tangency portfolio given a set of excess returns.

    Also, for convenience, this returns the associated vector of average
    returns and the variance-covariance matrix.

    Parameters
    ----------
    diagonalize_Sigma: bool
        When `True`, set the off diagonal elements of the variance-covariance
        matrix to zero.
    """
    Sigma = df_tilde.cov()
    # N is the number of assets
    N = Sigma.shape[0]
    Sigma_adj = Sigma.copy()
    if diagonalize_Sigma:
        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))

    mu_tilde = df_tilde.mean()
    Sigma_inv = np.linalg.inv(Sigma_adj)
    weights = Sigma_inv @ mu_tilde / (np.ones(N) @ Sigma_inv @ mu_tilde)
    # For convenience, I'll wrap the solution back into a pandas.Series object.
    omega_tangency = pd.Series(weights, index=mu_tilde.index)
    return omega_tangency, mu_tilde, Sigma

omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde)



In [45]:
def compute_gmv(df_tilde, diagonalize_Sigma=False):
    """Compute tangency portfolio given a set of excess returns.

    Also, for convenience, this returns the associated vector of average
    returns and the variance-covariance matrix.

    Parameters
    ----------
    diagonalize_Sigma: bool
        When `True`, set the off diagonal elements of the variance-covariance
        matrix to zero.
    """
    Sigma = df_tilde.cov()
    # N is the number of assets
    N = Sigma.shape[0]
    Sigma_adj = Sigma.copy()
    if diagonalize_Sigma:
        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))

    mu_tilde = df_tilde.mean()
    Sigma_inv = np.linalg.inv(Sigma_adj)
    weights = Sigma_inv @ np.ones(N) / (np.ones(N) @ Sigma_inv @ np.ones(N).T)
    # For convenience, I'll wrap the solution back into a pandas.Series object.
    omega_gmv = pd.Series(weights, index=mu_tilde.index)
    return omega_gmv, mu_tilde, Sigma

omega_gmv, mu_tilde, Sigma = compute_gmv(df_tilde)



In [46]:
np.ones(10)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Problem 2 (a)

In [47]:
omega_tangency

Fund A   -0.1102
Fund B    0.1164
Fund C   -0.0895
Fund D    0.5759
Fund E    0.5074
dtype: float64

In [48]:
omega_gmv

Fund A    0.1698
Fund B    0.2167
Fund C   -0.2256
Fund D    0.2413
Fund E    0.5977
dtype: float64

In [49]:
portfolio_stats(omega_tangency, mu_tilde, Sigma)

,Stat
Fraction in Risky Asset,1.0000
Mean,0.1215
Volatlity,0.0745
Sharpe Ratio,1.6304


In [50]:
portfolio_stats(omega_gmv, mu_tilde, Sigma)

,Stat
Fraction in Risky Asset,1.0000
Mean,0.1058
Volatlity,0.0695
Sharpe Ratio,1.5219


In [34]:
type(omega_tangency)

pandas.core.series.Series

In [6]:
omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde2)
omega_tangency

NameError: name 'df_tilde2' is not defined

In [54]:
df_tilde2 = df_tilde.dropna(axis=0)


In [60]:
df_tilde2.head()

,Domestic Equity,Foreign Equity,Emerging Markets,Private Equity,Absolute Return,High Yield,Commodities,Real Estate,Domestic Bonds,Foreign Bonds,Inflation-Indexed
Dates,,,,,,,,,,,
2009-03-31,0.0823,0.0828,0.1675,0.1523,-0.0123,0.0182,0.0439,0.0341,0.0320,0.0464,0.0579
2009-04-30,0.0988,0.1146,0.1550,0.2296,0.0223,0.1379,-0.0097,0.2956,-0.0280,0.0084,-0.0185
2009-05-29,0.0589,0.1324,0.1599,0.0544,0.0283,0.0290,0.1971,0.0232,-0.0203,0.0541,0.0205
2009-06-30,-0.0013,-0.0149,-0.0231,0.0448,-0.0040,0.0328,0.0051,-0.0257,-0.0061,0.0046,0.0014
2009-07-31,0.0746,0.1004,0.1102,0.1433,0.0154,0.0692,0.0045,0.1058,0.0083,0.0313,0.0009


In [126]:
ll=[]
for i in range(0,127):
    x = (df_tilde2.iloc[i,:] * omega_tangency).sum()
    ll.append(x)

In [127]:
df_optional= pd.DataFrame(ll,index=df_tilde2.index)

In [128]:
c=df_optional.mean()/df_optional.std()
c=c*np.sqrt(12)
c

0   2.3645
dtype: float64

In [124]:
0.07527387091111307*0.8743

0.06581194533758615

In [112]:
0.07527387091111307*c

0   0.0190
dtype: float64

In [129]:
for a in df_tilde2.columns:
    print(a,df_optional.iloc[:,0].corr(df_tilde[a])*c)

Domestic Equity 0   1.2063
dtype: float64
Foreign Equity 0   0.6185
dtype: float64
Emerging Markets 0   0.4779
dtype: float64
Private Equity 0   0.8201
dtype: float64
Absolute Return 0   0.5251
dtype: float64
High Yield 0   1.0645
dtype: float64
Commodities 0   -0.1053
dtype: float64
Real Estate 0   0.9988
dtype: float64
Domestic Bonds 0   0.6386
dtype: float64
Foreign Bonds 0   0.3292
dtype: float64
Inflation-Indexed 0   0.7423
dtype: float64


In [130]:
mu_tilde2 = 12 * df_tilde2.mean()
sigma2 = np.sqrt(12) * df_tilde2.std()
table2 = pd.DataFrame({'Mean':mu_tilde2, 'Vol':sigma2, 'Sharpe':mu_tilde2/sigma2})
table2

,Mean,Vol,Sharpe
Domestic Equity,0.1563,0.1296,1.2063
Foreign Equity,0.0975,0.1577,0.6185
Emerging Markets,0.0964,0.2017,0.4779
Private Equity,0.1637,0.1996,0.8201
Absolute Return,0.0232,0.0442,0.5251
High Yield,0.0882,0.0829,1.0645
Commodities,-0.0201,0.1909,-0.1053
Real Estate,0.1748,0.1750,0.9988
Domestic Bonds,0.0371,0.0581,0.6386
Foreign Bonds,0.0253,0.0769,0.3292


In [7]:

def portfolio_stats(omega, mu_tilde, Sigma, annualize_factor=12):
    mean = annualize_factor * mu_tilde @ omega
    vol = np.sqrt(annualize_factor) * np.sqrt(omega @ Sigma @ omega)
    sharpe_ratio = mean/vol
    df_stats = pd.DataFrame([omega.sum(), mean, vol, sharpe_ratio],
        index=['Fraction in Risky Asset', 'Mean', 'Volatlity', 'Sharpe Ratio'],
        columns=['Stat'])
    return df_stats

## Problem 2(b)

In [8]:
portfolio_stats(omega_tangency, mu_tilde, Sigma)

,Stat
Fraction in Risky Asset,1.0000
Mean,0.1251
Volatlity,0.0485
Sharpe Ratio,2.5799


## Problem 3

In [8]:


## Problem 3 ##

# (a) Compute and display the weights of MV portfolios with target returns...

def target_mv_portfolio(df_tilde, target_return=0.06, diagonalize_Sigma=False):
    """Compute MV optimal portfolio, given target return and set of excess returns.

    Parameters
    ----------
    diagonalize_Sigma: bool
        When `True`, set the off diagonal elements of the variance-covariance
        matrix to zero.
    """
    omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde, diagonalize_Sigma=diagonalize_Sigma)
    Sigma_adj = Sigma.copy()
    if diagonalize_Sigma:
        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))
    Sigma_inv = np.linalg.inv(Sigma_adj)
    N = Sigma_adj.shape[0]
    delta_tilde = ((np.ones(N) @ Sigma_inv @ mu_tilde)/(mu_tilde @ Sigma_inv @ mu_tilde)) * target_return
    omega_star = delta_tilde * omega_tangency
    print(target_return)
    return omega_star

omega_star = target_mv_portfolio(df_tilde, target_return=0.005)
print("Problem 3 (a)")
print(omega_star)

# Fraction in risky assets
omega_star.sum()

# (b) What is the mean, volatility, and Sharpe ratio for $w^p$?
print("Problem 3 (b)")
print(portfolio_stats(omega_star, mu_tilde, Sigma))


# (c) Discuss the allocation. In which asset is the portfolio most long? and short?
print("Problem 3 (c)")
print(omega_star.sort_values(ascending=False))

# (d) Does this line up with which assets have the strongest Sharpe ratios?
# See tables. It does not.
print("Problem 3 (d) - No")

0.005
Problem 3 (a)
Domestic Equity     -0.0539
Foreign Equity      -0.3226
Emerging Markets     0.2683
Private Equity       0.1263
Absolute Return      0.9483
High Yield           0.4227
Commodities         -0.0908
Real Estate         -0.1683
Domestic Bonds       0.9813
Foreign Bonds       -0.4952
Inflation-Indexed   -0.2162
dtype: float64
Problem 3 (b)
                          Stat
Fraction in Risky Asset 1.3998
Mean                    0.0600
Volatlity               0.0522
Sharpe Ratio            1.1495
Problem 3 (c)
Domestic Bonds       0.9813
Absolute Return      0.9483
High Yield           0.4227
Emerging Markets     0.2683
Private Equity       0.1263
Domestic Equity     -0.0539
Commodities         -0.0908
Real Estate         -0.1683
Inflation-Indexed   -0.2162
Foreign Equity      -0.3226
Foreign Bonds       -0.4952
dtype: float64
Problem 3 (d) - No


## Problem 4

In [20]:

## Problem 4 ##
# (a) Drop the inflation-indexed bonds from your return array, and recompute...

risky_assets_p4 = risky_assets.drop('Inflation-Indexed')
df_temp = df_tilde[risky_assets_p4]
omega_tangency, mu_tilde, Sigma = compute_tangency(df_temp)
omega_star = target_mv_portfolio(df_temp, target_return=0.005)
print("Problem 4(a)")
print(omega_star)

# (b) How does the portfolio compare to the allocation above...
# See written solutions

# (c) Calculate the Sharpe ratio. How much did it change?
print("Problem 4(c)")
print(portfolio_stats(omega_star, mu_tilde, Sigma))
# Sharpe ratio went down as expected, but not by very much.


0.005
Problem 4(a)
Domestic Equity    -0.0769
Foreign Equity     -0.3220
Emerging Markets    0.2659
Private Equity      0.1416
Absolute Return     1.0144
High Yield          0.4211
Commodities        -0.1067
Real Estate        -0.1879
Domestic Bonds      0.8798
Foreign Bonds      -0.5324
dtype: float64
Problem 4(c)
                          Stat
Fraction in Risky Asset 1.4969
Mean                    0.0600
Volatlity               0.0527
Sharpe Ratio            1.1382


## Problem 5 

In [10]:
## Problem 5 ##
# (a) Consider an allocation between only domestic and foreign equities.
risky_assets_p5 = ['Domestic Equity', 'Foreign Equity']
df_temp = df_tilde[risky_assets_p5].copy()
omega_star = target_mv_portfolio(df_temp, target_return=0.005)
print(omega_star)


# (b) ... See PDF
# (c) Make an adjustment
adjustment = df_temp['Foreign Equity'] + 0.001
df_temp['Foreign Equity'] = adjustment
omega_star = target_mv_portfolio(df_temp, target_return=0.005)
omega_star

0.005
Domestic Equity    1.0902
Foreign Equity    -0.0017
dtype: float64
0.005


Domestic Equity   0.4019
Foreign Equity    0.5740
dtype: float64

## Problem 6

In [11]:
## Problem 6 ##
omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde, diagonalize_Sigma=True)
omega_star = target_mv_portfolio(df_tilde, target_return=0.005, diagonalize_Sigma=True)
print(omega_star)
portfolio_stats(omega_star, mu_tilde, Sigma)

0.005
Domestic Equity     0.0784
Foreign Equity      0.0580
Emerging Markets    0.0699
Private Equity      0.0126
Absolute Return     0.3583
High Yield          0.1195
Commodities         0.0049
Real Estate         0.0795
Domestic Bonds      0.2982
Foreign Bonds       0.0783
Inflation-Indexed   0.2854
dtype: float64


,Stat
Fraction in Risky Asset,1.4428
Mean,0.0600
Volatlity,0.0873
Sharpe Ratio,0.6871


In [19]:

## Problem 7 ##
# (a) Using only data through the end of 2016, compute w^p for $\mu^p = .005$, ...
df_temp = df_tilde.loc[:'2016', :]
omega_tangency, mu_tilde, Sigma = compute_tangency(df_temp)
omega_star = target_mv_portfolio(df_temp, target_return=0.005)
print("Problem 7 (a)")
print(omega_star)

# (b) Calculate the portfolio's Sharpe ratio within that sample...
df_stats_in_sample = portfolio_stats(omega_star, mu_tilde, Sigma)
print("Problem 7 (b)")
print("in-sample performance for 2000-2016 using weights for period 2000-2016")
print(df_stats_in_sample)

# (c) Calculate the portfolio's Sharpe ratio based on performance in 2017-2019
df_temp = df_tilde.loc['2017':, :]
omega_tangency, mu_tilde_oos, Sigma_oos = compute_tangency(df_temp)
#omega_star = target_mv_portfolio(df_temp, target_return=0.005)
#print(omega_star)
df_stats_out_of_sample = portfolio_stats(omega_star, mu_tilde_oos, Sigma_oos)
print("Problem 7 (c)")
print("out-of-sample performance for 2017-2019 using weights for period 2000-2016")
print(df_stats_out_of_sample)


# (d) How does this out-of-sample Sharpe compare to the 2000-2016 performance
# of a portfolio optimized to $\mu^p$ using 2000-2016 data?

# See next question.

# (e) How does this out-of-sample Sharpe compare to the 2017-2019 performance...

# Peformance In-Sample and Out-of-Sample
omega_star_is = target_mv_portfolio(df_temp, target_return=0.005)
df_stats_in_sample_17_19 = portfolio_stats(omega_star_is, mu_tilde_oos, Sigma_oos)
(pd.DataFrame([
    df_stats_in_sample['Stat'],
    df_stats_out_of_sample['Stat'],
    df_stats_in_sample_17_19['Stat']
    ], index=['In-Sample: 2000-2016', 'Out-of-Sample', 'In-Sample: 2017-2019'])
    ).drop(columns=['Fraction in Risky Asset'])
print("Problem 7 (e)")
print(df_stats_in_sample_17_19)
# (f)

df_temp = df_tilde.loc[:'2016', :]
omega_tangency, mu_tilde, Sigma = compute_tangency(df_temp,diagonalize_Sigma=True)
omega_star = target_mv_portfolio(df_temp, target_return=0.005, diagonalize_Sigma=True)
df_stats_in_sample = portfolio_stats(omega_star, mu_tilde, Sigma)
#print(df_stats_in_sample)
# Calculate the portfolio's Sharpe ratio based on performance in 2017-2019
df_temp = df_tilde.loc['2017':, :]
omega_tangency, mu_tilde_oos, Sigma_oos = compute_tangency(df_temp)
df_stats_out_of_sample = portfolio_stats(omega_star, mu_tilde_oos, Sigma_oos)

# Peformance In-Sample and Out-of-Sample
omega_star_is = target_mv_portfolio(df_temp, target_return=0.005, diagonalize_Sigma=True)
df_stats_in_sample_17_19 = portfolio_stats(omega_star_is, mu_tilde_oos, Sigma_oos)
(pd.DataFrame([
    df_stats_in_sample['Stat'],
    df_stats_out_of_sample['Stat'],
    df_stats_in_sample_17_19['Stat']
    ], index=['In-Sample: 2000-2016', 'Out-of-Sample', 'In-Sample: 2017-2019'])
    ).drop(columns=['Fraction in Risky Asset'])


0.005
Problem 7 (a)
Domestic Equity     -0.0576
Foreign Equity      -0.1661
Emerging Markets     0.2171
Private Equity       0.0351
Absolute Return      0.7996
High Yield           0.2608
Commodities         -0.0551
Real Estate         -0.0881
Domestic Bonds       0.7260
Foreign Bonds       -0.5144
Inflation-Indexed   -0.0544
dtype: float64
Problem 7 (b)
in-sample performance for 2000-2016 using weights for period 2000-2016
                          Stat
Fraction in Risky Asset 1.1029
Mean                    0.0600
Volatlity               0.0457
Sharpe Ratio            1.3141
Problem 7 (c)
out-of-sample performance for 2017-2019 using weights for period 2000-2016
                          Stat
Fraction in Risky Asset 1.1029
Mean                    0.0204
Volatlity               0.0354
Sharpe Ratio            0.5761
0.005
Problem 7 (e)
                           Stat
Fraction in Risky Asset -1.1493
Mean                     0.0600
Volatlity                0.0210
Sharpe Ratio             

,Mean,Volatlity,Sharpe Ratio
In-Sample: 2000-2016,0.0600,0.0868,0.6916
Out-of-Sample,0.0487,0.0518,0.9402
In-Sample: 2017-2019,0.0600,0.0591,1.0150


In [16]:
df_temp = df_tilde.loc[:'2016', :]
omega_tangency, mu_tilde, Sigma = compute_tangency(df_temp,diagonalize_Sigma=True)
omega_star = target_mv_portfolio(df_temp, target_return=0.005, diagonalize_Sigma=True)
df_stats_in_sample = portfolio_stats(omega_star, mu_tilde, Sigma)
print(df_stats_in_sample)

0.005
                          Stat
Fraction in Risky Asset 1.3907
Mean                    0.0600
Volatlity               0.0868
Sharpe Ratio            0.6916


In [17]:
df_temp = df_tilde.loc['2017':, :]
omega_tangency, mu_tilde_oos, Sigma_oos = compute_tangency(df_temp)
#omega_star = target_mv_portfolio(df_temp, target_return=0.005)
#print(omega_star)
df_stats_out_of_sample = portfolio_stats(omega_star, mu_tilde_oos, Sigma_oos)


In [18]:
df_stats_out_of_sample

,Stat
Fraction in Risky Asset,1.3907
Mean,0.0487
Volatlity,0.0518
Sharpe Ratio,0.9402


In [33]:
df_temp = df_tilde.loc[:'2016', :]
omega_tangency, mu_tilde, Sigma = compute_tangency(df_temp)
omega_star = target_mv_portfolio(df_temp, target_return=0.005, diagonalize_Sigma=True)
df_stats_in_sample = portfolio_stats(omega_star, mu_tilde, Sigma)
print(df_stats_in_sample)
# Calculate the portfolio's Sharpe ratio based on performance in 2017-2019
df_temp = df_tilde.loc['2017':, :]
omega_tangency, mu_tilde_oos, Sigma_oos = compute_tangency(df_temp)
df_stats_out_of_sample = portfolio_stats(omega_star, mu_tilde_oos, Sigma_oos)

# Peformance In-Sample and Out-of-Sample
omega_star_is = target_mv_portfolio(df_temp, target_return=0.005, diagonalize_Sigma=True)
df_stats_in_sample_17_19 = portfolio_stats(omega_star_is, mu_tilde_oos, Sigma_oos)
(pd.DataFrame([
    df_stats_in_sample['Stat'],
    df_stats_out_of_sample['Stat'],
    df_stats_in_sample_17_19['Stat']
    ], index=['In-Sample: 2000-2016', 'Out-of-Sample', 'In-Sample: 2017-2019'])
    ).drop(columns=['Fraction in Risky Asset'])


0.005
                          Stat
Fraction in Risky Asset 1.3907
Mean                    0.0600
Volatlity               0.0868
Sharpe Ratio            0.6916
0.005


,Mean,Volatlity,Sharpe Ratio
In-Sample: 2000-2016,0.0600,0.0868,0.6916
Out-of-Sample,0.0487,0.0518,0.9402
In-Sample: 2017-2019,0.0600,0.0591,1.0150


In [118]:
import statistics
k=[2,3,5,67,8]
statistics.stdev(k)

28.044607324760317

In [122]:
k = [1-90,2-90,4-90,66-90,7-90]
statistics.stdev(k)

28.044607324760317